In [4]:
import os 
import sys 
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [13]:
with open('train.tsv', 'r') as f: 
    data = f.readlines()
    
with open('test.txt', 'r') as f: 
    test_data = f.readlines()

In [14]:
for i in range(len(data)): 
    data[i] = data[i].split('\t')
    data[i][0] = data[i][0].lower()
    data[i][1] = data[i][1].strip('\n').lower().split('-')
    
for i in range(len(test_data)): 
    test_data[i] = test_data[i].strip('\n').lower()

In [36]:
test_data[0]

'oradye'

In [16]:
label_pool = []
char_pool = [] 
for i in range(len(data)):
    label_pool+= data[i][1]
    char_pool += list(data[i][0].lower())
label_pool = list(set(label_pool))
char_pool = list(set(char_pool))




In [17]:
print(len(label_pool))
print(len(char_pool))

32
27


In [18]:
set(label_pool).difference(set(char_pool))

{'an', 'ch', 'en', 'ng', 'on', 'ou', 'oun', 'ui'}

In [19]:
def create_labels(data_list):
    single_tag = 0 
    double_tag = 0 
    triple_tag = 0
    label_list = [] 
    for i in range(len(data_list)):
        temp_labels = []
        for j in range(len(data_list[i][1])): 
            if len(data_list[i][1][j])==1:
                temp_labels += ['B']
                single_tag += 1 
            elif len(data_list[i][1][j])==2:
                temp_labels += ['B','I']
                double_tag += 1 
            elif len(data_list[i][1][j])==3:
                temp_labels += ['B','I','I']
                triple_tag += 1 
        label_list.append(temp_labels)
    print("The number of single tags are: ", single_tag)
    print("The number of double tags are: ", double_tag)
    print("The number of triple tags are: ", triple_tag)
    return label_list

In [20]:
labels = create_labels(data)

The number of single tags are:  72096
The number of double tags are:  7592
The number of triple tags are:  8


In [21]:
encoded_label_pool = []
for e in labels:
    encoded_label_pool+= e

encoded_label_pool = list(set(encoded_label_pool))


In [53]:
def pad(char_list, max_len = 18):
    if max_len==None: 
        max_len = 0 
        for i in range(len(char_list)):
            if len(char_list[i])>max_len: 
                max_len = len(char_list[i])
    for i in range(len(char_list)): 
        if len(char_list[i])< max_len: 
            for _ in range(max_len- len(char_list[i])): 
                char_list[i] += ['<PAD>']
    return char_list

In [54]:
def encode_words(char_list, char_dict):
    encoded_chars = [] 
    for i in range(len(char_list)):
        temp_encoded = [] 
        for j in range(len(char_list[i])):
            temp_encoded.append(char_dict[char_list[i][j]]) 
        encoded_chars.append(np.array(temp_encoded))   # .reshape(-1,1)
    return encoded_chars

In [55]:
label_dict = {}
label_dict['<PAD>'] = 0
counter = 1 
for e in encoded_label_pool: 
    label_dict[e] = counter 
    counter += 1 
    
invers_label_dict = {v:k for k,v in label_dict.items()}

In [56]:
invers_label_dict

{0: '<PAD>', 1: 'B', 2: 'I'}

In [57]:
padded_labels = pad(labels)

In [58]:
encoded_labels  = encode_words(padded_labels, label_dict)

In [59]:
encoded_labels[0]

array([1, 1, 2, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [60]:
char_dict = {}
char_dict['<PAD>'] = 0
char_dict['<UNK>'] = 1 
counter = 2 
for e in char_pool: 
    char_dict[e] = counter 
    counter += 1 

In [97]:
invers_char_dict = {v:k for k,v in char_dict.items()}

char_list = []
for e in data: 
    char_list.append(list(e[0]))

In [62]:
test_char_list = []
for e in test_data: 
    test_char_list.append(list(e))

In [63]:
test_char_list[1]

['b', 'o', 'u', 's']

In [64]:
char_list[0]

['k', 'o', 'n', 's', 'i', 'l', 't', 'a', 'n']

In [65]:
paded_list = pad(char_list)
padded_test_list = pad(test_char_list)

In [66]:
encoded_data = encode_words(paded_list, char_dict)
encoded_test_data = encode_words(padded_test_list, char_dict)

In [67]:
encoded_data[0:3]

[array([21, 12,  2,  9,  4, 23, 18, 28,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0]),
 array([14, 22, 20, 12, 18, 19, 28,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0]),
 array([ 9, 12,  9, 10, 12, 20, 19, 12, 13, 22,  9, 10, 12,  2,  5, 23,  0,
         0])]

In [68]:
X_train, X_test, y_train, y_test = train_test_split(np.array(encoded_data), np.array(encoded_labels), test_size=0.1, random_state=42)

final_test = np.array(encoded_test_data)

In [69]:
class Masked_Accuracy(tf.keras.metrics.SparseCategoricalAccuracy):
    def __init__(self,):
        super(Masked_Accuracy, self).__init__()

    def call(self, y_true, y_pred):
        total = 0.0
        for i in range(len(y_true)):
            y_t = y_true[i][y_true[i]!=0]
            y_p = tf.argmax(y_pred[i][:len(y_t)], axis = -1)
            self.update_state(y_t, y_p)
            total += m.result()
        return total/tf.cast(len(y_true), tf.float32)

In [70]:
def masked_accuracy(y_true, y_pred):
    total = 0.0
    for i in range(len(y_true)):
        y_t = y_true[i][y_true[i]!=0]
        y_p = tf.argmax(y_pred[i], axis = -1)[:len(y_t)]
        m = tf.keras.metrics.Accuracy()
        m.update_state(y_t, y_p)
        total += m.result()
    return total/tf.cast(len(y_true), tf.float32)
    

In [45]:
input_chars_input = len(char_dict)
embedding_vector_length = 16

inputs = tf.keras.Input(shape=(X_train.shape[-1]), name="word_input")
x = tf.keras.layers.Embedding(input_chars_input, embedding_vector_length, input_length = 1)(inputs)
print(x.shape)

# x = tf.keras.layers.LSTM(128,return_sequences=True)(x) # 128
# x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True))(x) # 128
print(x.shape)
x = tf.keras.layers.Dropout(0.2)(x)

# dense_layer = tf.keras.layers.Dense(64, activation="tanh", name="dense_1")
# x = tf.keras.layers.TimeDistributed(dense_layer)(x)

output_layer = tf.keras.layers.Dense(3, activation="softmax", name="predictions")
outputs = tf.keras.layers.TimeDistributed(output_layer)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


# recall = tf.keras.metrics.Recall(class_id=4)
scce = tf.keras.metrics.SparseCategoricalCrossentropy()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[Masked_Accuracy(),scce]) # recall, sparse_categorical_cross_entropy
print(model.summary())

model.fit(X_train, y_train, validation_split =0.2, epochs=10, batch_size=32)

scores = model.evaluate(X_test, y_test, verbose=0)
print("The final accuracy in test set is: %.2f%%" % (scores[1]*100))


2022-10-11 17:51:52.504733: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-11 17:52:00.512842: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4562 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:1c:00.0, compute capability: 8.0
2022-10-11 17:52:00.546306: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 11725 MB memory:  -> device: 1, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:1d:00.0, compute capability: 8.0
2022-10-11 17:52:00.546899: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/re

(None, 18, 16)
(None, 18, 256)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 word_input (InputLayer)     [(None, 18)]              0         
                                                                 
 embedding (Embedding)       (None, 18, 16)            464       
                                                                 
 bidirectional (Bidirectiona  (None, 18, 256)          148480    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 18, 256)           0         
                                                                 
 time_distributed (TimeDistr  (None, 18, 3)            771       
 ibuted)                                                         
                                                                 
Total params: 149,715
Trainabl

2022-10-11 17:52:12.573170: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8200
2022-10-11 17:52:16.205656: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


289/289 [==============================] - 18s 14ms/step - loss: 0.1989 - sparse_categorical_accuracy: 0.9318 - sparse_categorical_crossentropy: 0.1989 - val_loss: 0.0355 - val_sparse_categorical_accuracy: 0.9881 - val_sparse_categorical_crossentropy: 0.0355
Epoch 2/10
289/289 [==============================] - 4s 13ms/step - loss: 0.0221 - sparse_categorical_accuracy: 0.9922 - sparse_categorical_crossentropy: 0.0221 - val_loss: 0.0116 - val_sparse_categorical_accuracy: 0.9974 - val_sparse_categorical_crossentropy: 0.0116
Epoch 3/10
289/289 [==============================] - 4s 13ms/step - loss: 0.0093 - sparse_categorical_accuracy: 0.9977 - sparse_categorical_crossentropy: 0.0093 - val_loss: 0.0057 - val_sparse_categorical_accuracy: 0.9987 - val_sparse_categorical_crossentropy: 0.0057
Epoch 4/10
289/289 [==============================] - 3s 11ms/step - loss: 0.0051 - sparse_categorical_accuracy: 0.9991 - sparse_categorical_crossentropy: 0.0051 - val_loss: 0.0040 - val_sparse_categoric

In [46]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("The final accuracy in test set is: %.2f%%" % (scores[1]*100))


The final accuracy in test set is: 99.95%


In [151]:
def decode(y_true, y_pred, inverse_dictionary, sparse = True):
    decoded_list = [] 
    for i in range(len(y_pred)): 
        temp_len = len(y_true[i][y_true[i]!=0])
        temp_prediction = y_pred[i]
        if sparse: 
            temp_prediction = tf.argmax(temp_prediction, axis = -1).numpy()
        temp_prediction = temp_prediction[:temp_len]
        temp_list = [] 
        for j in range(len(temp_prediction)):
            temp_list.append(inverse_dictionary[temp_prediction[j]])
        decoded_list.append(temp_list)
    return decoded_list

In [152]:
predicted_test = model.predict(X_test)

41/41 [==============================] - 0s 4ms/step


In [153]:
decoded_predictions = decode(y_test, predicted_test, invers_label_dict)

In [154]:
decoded_labels = decode(y_test, y_test, invers_label_dict, sparse = False )

In [155]:
print("The predictions for the first sample is as follows: \n", decoded_predictions[0])
print("The labels for the first sample is as follows: \n", decoded_labels[0])

The predictions for the first sample is as follows: 
 ['B', 'B', 'B', 'B', 'I', 'B', 'B', 'B', 'B']
The labels for the first sample is as follows: 
 ['B', 'B', 'B', 'B', 'I', 'B', 'B', 'B', 'B']


In [156]:
def precision(y_true, y_pred, tp_label , sparse = True):
    decoded_list = [] 
    tp = 0 
    all_pred_p = 0 
    for i in range(len(y_pred)): 
        temp_len = len(y_true[i][y_true[i]!=0])
        temp_prediction = y_pred[i]
        if sparse: 
            temp_prediction = tf.argmax(temp_prediction, axis = -1).numpy()
        temp_prediction = temp_prediction[:temp_len]
        label = y_test[i][:temp_len]
        tp += np.sum(np.multiply(label==tp_label, temp_prediction==tp_label))
        all_pred_p += np.where(temp_prediction==tp_label)[0].shape[0]
        
    return tp/(all_pred_p + np.finfo(float).eps)


def recall(y_true, y_pred, tp_label , sparse = True):
    decoded_list = [] 
    tp = 0
    all_p = 0 
    for i in range(len(y_pred)): 
        temp_len = len(y_true[i][y_true[i]!=0])
        temp_prediction = y_pred[i]
        if sparse: 
            temp_prediction = tf.argmax(temp_prediction, axis = -1).numpy()
        temp_prediction = temp_prediction[:temp_len]
        label = y_test[i][:temp_len]
        tp += np.sum(np.multiply(label==tp_label, temp_prediction==tp_label))
        all_p += label[label==tp_label].shape[0]
        
    return tp/(all_p + np.finfo(float).eps)


def f1(y_true, y_pred, tp_label):
    '''
    y_true: showing the true labels 
    y_pred: showing the predicted values
    tp_label: showing the p
        p: precison 
        r: recall 
        
    '''
    p = precision(y_true, y_pred, tp_label)
    r = recall(y_true, y_pred, tp_label)
    return 2 * ((p * r) / (p + r +  np.finfo(float).eps))

In [157]:
f1(y_test, predicted_test, 1)

0.9993114241001564

In [158]:
print("The f1 score based on \"I\" tags is: %.5f"%(f1(y_test, predicted_test, 1)*100))

The f1 score based on "I" tags is: 99.93114


## Testing the test file over the current Model

In [159]:
predicted_test = model.predict(final_test)

45/45 [==============================] - 0s 4ms/step


In [161]:
decoded_predictions = decode(final_test, predicted_test, invers_label_dict, sparse = True )

In [162]:
def decode_mutations(X, y, X_dict, y_dict): 
    output = [] 
    for i in range(len(X)):
        temp_str = ''
        temp_output = []
        temp_X = X[i][X[i]!=0]
        for j in range(len(temp_X)):
            if y[i][j] == 'B':
                temp_str += X_dict[temp_X[j]]
                temp_output.append(temp_str)
                temp_str = ''
            elif y[i][j] == 'I':
                temp_output[-1] += X_dict[temp_X[j]]
            
        output.append('-'.join(temp_output))
        
    return output 

In [163]:
output = decode_mutations(final_test,decoded_predictions, invers_char_dict, label_dict)

In [168]:
with open("supervised.tsv", 'w') as f: 
    pass
for i in range(len(output)): 
    with open("supervised.tsv", 'a+') as f: 
        f.write(test_data[i]+'\t'+output[i]+'\n')

In [170]:
output[i]

't-e-l-e-k-o-p-i'